In [2]:
# Install and import dependencies
!pip install gradio --quiet

import pandas as pd
import re
import requests
import gradio as gr
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from google.colab import files
uploaded = files.upload()

# Load training data from CSV (uploaded to Colab)
df = pd.read_csv("training_data.csv")  # Update path if needed

# Assuming your CSV has a column named 'text,label' containing both text and label
# Split the column into two, handling potential extra commas by limiting to 2 splits
df[['text', 'label']] = df['text,label'].str.split(',', n=1, expand=True)  # Limit to 1 split

train_texts = df['text'].tolist()
train_labels = df['label'].tolist()

# TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(train_texts)

# Model training
model = LogisticRegression()
model.fit(X_train, train_labels)

# Chatbot class
class Chatbot:
    def __init__(self):
        self.memory = {}
        self.conversation_mode = 'casual'

    def extract_city(self, user_input):
        cleaned = re.sub(r"\b(how|why|what|describe|explain|when|now|please|tell|is|the|in|on|show|me|about|current|like|of|to|can|you|today|weather|good|chill|hot|nice|bad|condition)\b", "", user_input.lower())
        cleaned = re.sub(r"\s+", " ", cleaned).strip()
        cleaned = re.sub(r"[^a-zA-Z\s]", "", cleaned)
        return cleaned if cleaned else None

    def get_weather(self, city_name):
        api_key = "88561f09dedb074b3405971aa2ad1342"  # Replace with your API key
        url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}&units=metric"
        response = requests.get(url)
        data = response.json()

        if data.get("cod") != 200:
            return "City not found. Please check the name."

        weather = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        feels_like = data["main"]["feels_like"]
        humidity = data["main"]["humidity"]

        return (f"Weather in {city_name.title()}:\n"
                f"\U0001F324 {weather.title()}\n"
                f"\U0001F321 Temperature: {temperature}°C\n"
                f"\U0001F975 Feels Like: {feels_like}°C\n"
                f"\U0001F4A7 Humidity: {humidity}%")

    def generate_response(self, user_input):
      user_input_lower = user_input.lower()
      if "formal" in user_input_lower: # Corrected indentation
        self.conversation_mode = 'formal'
        return "Switched to formal mode."
      elif "casual" in user_input_lower: # Corrected indentation
        self.conversation_mode = 'casual'
        return "Switched to casual mode."
      elif "tech support" in user_input_lower: # Corrected indentation
        self.conversation_mode = 'tech support'
        return "Switched to tech support mode."


    # Predict label
      X_input = vectorizer.transform([user_input_lower])
      predicted_label = model.predict(X_input)[0]

    # Respond based on label
      if predicted_label == "weather":
          city = self.extract_city(user_input)
          if city:
              return self.get_weather(city)
          else:
              return "Please specify a city for the weather."

      elif predicted_label == "tech":
          if "wifi" in user_input_lower or "network" in user_input_lower:
              return random.choice([
                  "Try restarting your router and checking other devices.",
                  "Check if other devices are facing the same issue.",
                  "Turn your modem off and on again, then reconnect."
              ])
          elif "python" in user_input_lower or "error" in user_input_lower:
              return random.choice([
                  "Share the error message and I’ll debug it.",
                  "Can you paste the Python error here?",
                  "Let me know what the error says exactly."
              ])
          elif "laptop" in user_input_lower or "system" in user_input_lower:
             return random.choice([
                  "Is your laptop freezing or showing any error?",
                  "Try closing unused apps and restarting.",
                  "Check for background processes slowing it down."
             ])
          else:
              return "Can you describe the technical issue more clearly?"

      elif predicted_label == "casual":
          casual_responses = [
              "Hey! That’s cool 😎. What's up?",
              "Yo! What's happening?",
              "Haha, that sounds fun!",
              "Vibe check passed 😄"
          ]
          formal_responses = [
              "Understood. Could you please elaborate further?",
              "Noted. Please provide more context.",
              "Thank you. Could you explain that in detail?",
              "Okay. Would you mind clarifying your point?"
          ]
          if self.conversation_mode == "casual":
              return random.choice(casual_responses)
          elif self.conversation_mode == "formal":
              return random.choice(formal_responses)

      return "I'm not sure how to respond to that."

# Gradio interface
chatbot = Chatbot()

def respond_to_user(user_input):
    return chatbot.generate_response(user_input)

gr.Interface(fn=respond_to_user, inputs="text", outputs="text", title="Weather & Tech Chatbot").launch()


Saving training_data.csv to training_data (1).csv
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://92e87bf75701f13060.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
